<h1 align=center><font size = 5 >Capstone Project | Explore Segmenting and cluster the neighborhood of | Delhi | India </font></h1>

## Table of content

* 1. Importing all required Libraries
* 2. Scrap Data from wikipedia page into a DataFrame
* 3. Get the Geographical Coordinates
* 4. Create map of Delhi using Latitude and longitude
* 5. Use the Foursquare API to explore the neighborhoods
* 6. Analyze Each Neighborhood
* 7. Examine clusters

## 1.Importing all required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Required Libraries imported.")

Required Libraries imported.


## 2. Scrap Data from wikipedia page into a DataFrame

In [20]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Delhi_geography_stubs").text

In [21]:
soup = BeautifulSoup(data, 'html.parser')

In [22]:
neighborhoodList = []

In [23]:
for row in soup.find_all("div", class_ = "mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [24]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head(50)

,Neighborhood
0,Template:Delhi-geo-stub
1,Aastha Kunj
2,Ajmal Khan Park
3,"Alipur, Delhi"
4,Asalat Pur Khawad
5,Ashram Chowk
6,Asola (Delhi)
7,Azadpur
8,Babar Pur
9,Bagdola


In [25]:
df.shape

(161, 1)

## 3. Get the geographical coordinates

In [27]:
# ddefine a function to get coordinates
def get_latlng(neighborhood):
    #initialize your varialbe to none
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi , India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [28]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords

[[28.63409000000007, 77.21693000000005],
 [28.554730000000063, 77.25506000000007],
 [28.653400000000033, 77.19739000000004],
 [28.798050000000046, 77.14426000000003],
 [28.541890000000024, 76.95024000000006],
 [28.710598435255907, 77.32696519316737],
 [28.451500000000067, 77.21149000000008],
 [28.712420000000066, 77.17304000000007],
 [28.68219848827012, 77.28533883577524],
 [25.07095000000004, 73.91949000000005],
 [28.820740000000058, 77.19310000000007],
 [28.554890000000057, 77.16654000000005],
 [28.75159000000008, 79.44737000000003],
 [28.853040000000078, 77.07615000000004],
 [28.639220000000023, 77.01251000000008],
 [28.73894000000007, 77.23745000000008],
 [28.764760000000024, 77.06616000000008],
 [28.732620000000054, 77.05213000000003],
 [28.629200000000026, 77.23215000000005],
 [28.74508000000003, 77.15873000000005],
 [28.442900000000066, 77.18147000000005],
 [28.828580000000045, 77.08225000000004],
 [28.582486353036153, 77.24835460432588],
 [28.55782000000005, 77.00172000000003],

In [29]:
# Create temporary dataFrame to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns = ['Latitude', 'Longitude'])

In [30]:
# merge the coordiantes into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [31]:
print(df.shape)
df

(161, 3)


,Neighborhood,Latitude,Longitude
0,Template:Delhi-geo-stub,28.634090,77.216930
1,Aastha Kunj,28.554730,77.255060
2,Ajmal Khan Park,28.653400,77.197390
3,"Alipur, Delhi",28.798050,77.144260
4,Asalat Pur Khawad,28.541890,76.950240
5,Ashram Chowk,28.710598,77.326965
6,Asola (Delhi),28.451500,77.211490
7,Azadpur,28.712420,77.173040
8,Babar Pur,28.682198,77.285339
9,Bagdola,25.070950,73.919490


In [33]:
# Save the data set for further use
df.to_csv('Delhi_df.csv', index=False)

## 4. create map of Delhi Using Latitude and longitude

In [34]:
# get the coordinates 
address = 'Delhi, India'

geolocator = Nominatim(user_agent = "my-applications")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The Loaction in Delhi , India {}, {}.".format(latitude, longitude))

The Loaction in Delhi , India 28.6517178, 77.2219388.


In [37]:
map_Dl = folium.Map(location = [latitude, longitude], zoom_start = 13)

# add marker in the map of Delhi
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], 
                        radius = 6, 
                        popup = label, 
                        color= 'Purple', 
                        fill=True, 
                        fill_color = 'red', 
                        fill_opacity= 0.6).add_to(map_Dl)

In [38]:
map_Dl

**Save the map as HTML file**

In [40]:
map_Dl.save('map_Delhi.html')

## 5. Use the Foursquare API to explore the neighborhood

In [41]:
CLIENT_ID = '0ZOFFGKXDEMNTGR1IE1TZCCY04OLYB3JP4P1S2LFWIKMEKAG'
CLIENT_SECRET = '5FDLAH5HHGINTCNW5GXYGXDR21WEAOBDG4IC3FES22KXD0KW'
VERSION = '20180604'

print('YOUR CRIDENTIALS')
print('CLIENT_ID : ', CLIENT_ID)
print('CLIENT_SECRET : ', CLIENT_SECRET)
print('VERSION: ', VERSION)

YOUR CRIDENTIALS
CLIENT_ID :  0ZOFFGKXDEMNTGR1IE1TZCCY04OLYB3JP4P1S2LFWIKMEKAG
CLIENT_SECRET :  5FDLAH5HHGINTCNW5GXYGXDR21WEAOBDG4IC3FES22KXD0KW
VERSION:  20180604


**Now Let's get the top venues**

In [42]:
radius  = 5000 # 5 km
LIMIT = 200
venues = []

In [44]:
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # Create the API request URL from FOURSQUARE
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT
    )
    
    
    # for get request 
    results = requests.get(url).json()["response"]["groups"][0]["items"]
    
    # retuen only relevant information for each nearby venue
    for venue in results:
        venues.append((
        neighborhood,
        lat,
        lng,
        venue['venue']['name'],
        venue['venue']['location']['lat'],
        venue['venue']['location']['lng'],
        venue['venue']['categories'][0]['name']))

In [45]:
venue_df = pd.DataFrame(venues)

In [46]:
venue_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']

In [47]:
venue_df.shape

(7724, 7)

In [48]:
venue_df.head()

,Neighborhood,Latitude,Longitude,Venue Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Template:Delhi-geo-stub,28.63409,77.21693,Connaught Place | कनॉट प्लेस (Connaught Place),28.632731,77.220018,Plaza
1,Template:Delhi-geo-stub,28.63409,77.21693,Wenger's,28.633412,77.218292,Bakery
2,Template:Delhi-geo-stub,28.63409,77.21693,Naturals Ice Cream,28.634455,77.222139,Ice Cream Shop
3,Template:Delhi-geo-stub,28.63409,77.21693,Khan Chacha | खान चाचा | خان چاچا,28.634202,77.220780,Indian Restaurant
4,Template:Delhi-geo-stub,28.63409,77.21693,The Imperial,28.625548,77.218664,Hotel


In [50]:
venue_df['Neighborhood'].count()

7724

In [51]:
venue_df.groupby(['Neighborhood']).count()

,Latitude,Longitude,Venue Name,Venue_Latitude,Venue_Longitude,Venue_Category
Neighborhood,,,,,,
Aastha Kunj,100,100,100,100,100,100
Ajmal Khan Park,100,100,100,100,100,100
"Alipur, Delhi",4,4,4,4,4,4
Asalat Pur Khawad,1,1,1,1,1,1
Ashram Chowk,11,11,11,11,11,11
Asola (Delhi),5,5,5,5,5,5
Azadpur,84,84,84,84,84,84
Babar Pur,29,29,29,29,29,29
Bagdola,2,2,2,2,2,2


In [54]:
venue_df['Venue_Category'].unique()[:100]

array(['Plaza', 'Bakery', 'Ice Cream Shop', 'Indian Restaurant', 'Hotel',
       'Coffee Shop', 'South Indian Restaurant', 'Café',
       'Molecular Gastronomy Restaurant', 'Food Truck', 'Bar',
       'Clothing Store', 'Restaurant', 'Chinese Restaurant', 'Lounge',
       'Bistro', 'Smoke Shop', 'Portuguese Restaurant',
       'Tibetan Restaurant', 'Spa', 'Theater', 'Flea Market',
       'Asian Restaurant', 'Monument / Landmark', 'Arcade', 'Art Gallery',
       'Sculpture Garden', 'Snack Place', 'Art Museum', 'Historic Site',
       'Music Venue', 'Italian Restaurant', 'Boutique', 'Market',
       'Mediterranean Restaurant', 'Food & Drink Shop', 'Irani Cafe',
       'Dessert Shop', 'Mosque', 'Golf Course', 'French Restaurant',
       'Park', 'History Museum', 'Japanese Restaurant', 'Bookstore',
       'Fast Food Restaurant', 'Sandwich Place', 'Furniture / Home Store',
       'Karnataka Restaurant', 'BBQ Joint', 'Northeast Indian Restaurant',
       'Food Court', 'Temple', 'Indie Movie T

In [55]:
"Neighborhood" in venue_df['Venue_Category'].unique()

True

## 6. Analyze Each Neighborhood

In [87]:
# one hot encoding
Dl_onehot = pd.get_dummies(venue_df[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Dl_onehot['Neighborhoods'] = venue_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Dl_onehot.columns[-1]] + list(Dl_onehot.columns[:-1])
Dl_onehot = Dl_onehot[fixed_columns]

print(Dl_onehot.shape)
Dl_onehot.head()

(7724, 189)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motel,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Nightlife,Palace,Park,Performing Arts Venue,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Travel Agency,Travel Lounge,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Template:Delhi-geo-stub,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Template:Delhi-geo-stub,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Template:Delhi-geo-stub,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Template:Delhi-geo-stub,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [88]:
print(Dl_onehot.shape)

(7724, 189)


In [94]:
Dl_grouped = Dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Dl_grouped.shape)
Dl_grouped

(154, 189)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motel,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Nightlife,Palace,Park,Performing Arts Venue,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Travel Agency,Travel Lounge,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Aastha Kunj,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.020000,0.00,0.00,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.040000,0.00,0.030000,0.000000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.040000,0.000000,0.010000,0.000000,0.010000,0.01,0.00,0.00,0.000000,0.00,0.050000,0.010000,0.0,0.010000,0.000000,0.010000,0.000000,0.01,0.000000,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.050000,0.01,0.000000,0.020000,0.00,0.120000,0.000000,0.010000,0.00,0.050000,0.02,0.00,0.000000,0.00,0.00,0.00,0.0,0.000000,0.040000,0.060000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.060000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.020000,0.00,0.010000,0.01,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.

In [96]:
len(Dl_grouped[Dl_grouped["Shopping Mall"] > 0])

65

In [99]:
# Dl_SM = Delhi_Shopping_Mall
Dl_SM = Dl_grouped[['Neighborhoods', 'Shopping Mall']]

In [100]:
Dl_SM.head()

,Neighborhoods,Shopping Mall
0,Aastha Kunj,0.02
1,Ajmal Khan Park,0.00
2,"Alipur, Delhi",0.00
3,Asalat Pur Khawad,0.00
4,Ashram Chowk,0.00


## 7. Cluster Neighborhoods

Run k-means to the cluster the neighborhood in Delhi in 4 clusters

In [101]:
# set number of clusters
kclusters = 4

In [102]:
Dl_clustering  = Dl_SM.drop(["Neighborhoods"], 1)

In [103]:
# run the k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(Dl_clustering)

In [104]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([3, 0, 0, 0, 0, 0, 3, 0, 0, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0])

In [105]:
# create a new dataframe that includes the cluster as well as the top venues for each neighborhood
Dl_merged = Dl_SM.copy()

In [106]:
# add clustering labels
Dl_merged["Clusters Labels"] = kmeans.labels_

In [109]:
Dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Dl_merged.head()

,Neighborhood,Shopping Mall,Clusters Labels
0,Aastha Kunj,0.02,3
1,Ajmal Khan Park,0.00,0
2,"Alipur, Delhi",0.00,0
3,Asalat Pur Khawad,0.00,0
4,Ashram Chowk,0.00,0


In [110]:
# merge Dl_grouped with df coords_df to add latitude and longitu for each neighborhood
Dl_merged = Dl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")



In [112]:
print(Dl_merged.shape)

(154, 5)


In [113]:
Dl_merged.head()

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
0,Aastha Kunj,0.02,3,28.554730,77.255060
1,Ajmal Khan Park,0.00,0,28.653400,77.197390
2,"Alipur, Delhi",0.00,0,28.798050,77.144260
3,Asalat Pur Khawad,0.00,0,28.541890,76.950240
4,Ashram Chowk,0.00,0,28.710598,77.326965


In [117]:
# sort the results by Cluster Labels
print(Dl_merged.shape)
Dl_merged.sort_values(["Clusters Labels"], inplace=True)
Dl_merged

(154, 5)


,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
153,Ziauddin Pur,0.000000,0,28.703440,77.276130
73,Lodhi Colony,0.000000,0,28.584750,77.225340
114,Pusa Road,0.000000,0,28.644450,77.187660
71,Kirari Suleman Nagar,0.000000,0,28.694510,77.044560
70,Khizrabad Village,0.010000,0,28.571670,77.273170
69,Khera Khurd,0.000000,0,28.776460,77.099740
68,Khera Kalan,0.000000,0,28.774180,77.123410
67,Khera Dabar,0.000000,0,28.584030,76.936020
66,Kharkhari Nahar,0.000000,0,28.586170,76.959640
65,KharKhari Jatmal,0.000000,0,28.579250,76.945200


**Finally , Let's Visualize resulting clusters**

In [119]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 13)

# set the color scheme for thr clusters
X = np.arange(kclusters)
Y = [i+X+(i*X)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(Y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [120]:
# add makekers to the map of Delhi
markers_colors = []
for lat, lng, poi, cluster in zip(Dl_merged['Latitude'], Dl_merged['Longitude'], Dl_merged['Neighborhood'], Dl_merged['Clusters Labels']):
    label = folium.Popup(str(poi) + ' - Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 6,
    popup = label,
    color = rainbow[cluster-1],
    fill = True,
    fill_color = rainbow[cluster-1],
    fill_opacity = 0.7).add_to(map_clusters)

In [121]:
map_clusters

In [122]:
# save the map in HMTL file
map_clusters.save('map_clusters_Delhi.html')

## 8. Examine clusters

* Cluster = 0

In [123]:
Dl_merged.loc[Dl_merged['Clusters Labels'] == 0]

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
153,Ziauddin Pur,0.00,0,28.703440,77.276130
73,Lodhi Colony,0.00,0,28.584750,77.225340
114,Pusa Road,0.00,0,28.644450,77.187660
71,Kirari Suleman Nagar,0.00,0,28.694510,77.044560
70,Khizrabad Village,0.01,0,28.571670,77.273170
69,Khera Khurd,0.00,0,28.776460,77.099740
68,Khera Kalan,0.00,0,28.774180,77.123410
67,Khera Dabar,0.00,0,28.584030,76.936020
66,Kharkhari Nahar,0.00,0,28.586170,76.959640
65,KharKhari Jatmal,0.00,0,28.579250,76.945200


* clusters = 1

In [125]:
Dl_merged.loc[Dl_merged['Clusters Labels'] == 1]

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
123,Sahibabad Daulat Pur,0.25,1,28.753010,77.074210
139,Taj Pul,0.25,1,28.839135,77.142675
38,"Hamidpur, Delhi",0.20,1,28.832490,77.139140
10,Bankner,0.25,1,28.853040,77.076150
144,Tikri Khurd,0.25,1,28.834710,77.116680


* Cluster = 2

In [126]:
Dl_merged.loc[Dl_merged['Clusters Labels'] == 2]

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
131,Shankar Vihar,0.088889,2,28.566400,77.140330
134,Siras Pur,0.083333,2,28.761530,77.133050
147,"Vasant Vihar, Delhi",0.050000,2,28.565210,77.161150
118,Rajokri,0.060000,2,28.519370,77.121260
129,Shadipur Depot,0.044118,2,28.721810,77.164520
105,"Outer Ring Road, Delhi",0.074074,2,28.693168,77.327142
111,Pooth Kalan,0.055556,2,28.714350,77.076900
110,Pehlad Pur Bangar,0.090909,2,28.744050,77.080109
76,Mehram nagar,0.050000,2,28.566310,77.128340
32,"Gadaipur, Mehrauli, New Delhi",0.050000,2,28.521130,77.180170


* cluster = 3

In [128]:
Dl_merged.loc[Dl_merged['Clusters Labels'] == 3]

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude
99,North Delhi Municipal Corporation,0.031579,3,28.709560,77.179250
135,South Delhi Municipal Corporation,0.030000,3,28.658610,77.114440
136,South East Delhi district,0.020000,3,28.550650,77.251870
137,Subhash vihar,0.037037,3,28.697070,77.268890
31,East Delhi Municipal Corporation,0.037037,3,28.627960,77.303040
30,Dholpur House,0.033333,3,28.695650,77.100910
28,Derawal Nagar,0.025316,3,28.699110,77.191050
143,Tihar Village,0.040000,3,28.635470,77.107780
142,Tigri,0.020000,3,28.510210,77.237450
25,Dayanand Colony,0.020000,3,28.560590,77.246780


* clusters = 4

In [129]:
Dl_merged.loc[Dl_merged['Clusters Labels'] == 4]

,Neighborhood,Shopping Mall,Clusters Labels,Latitude,Longitude


## Conclusions

Most of the shopping malls are concentrated in the Delhi where cluster lables is 3 . On the other hand, cluster 2 & 1 has very low number to totally no shopping mall in the neighborhoods

This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 3 are likely suffering from intense competition due to oversupply and high concentration of shopping malls.

## Thank you !